In [35]:
!pip install -qU torchvision

In [36]:
!pip install pillow

In [37]:
!pip install requests

# MNIST Training using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

In [3]:
import json
metadata = json.load(open("/opt/ml/metadata/resource-metadata.json","r"))
bucket = metadata["UserProfileName"]
bucket

'1cd945b4-090e-4741-982f-318c9ed188c5'

In [4]:
bucket

'1cd945b4-090e-4741-982f-318c9ed188c5'

In [5]:
from boto3 import client

conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
for key in conn.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])

data/1.10.109.6347.21761.858772.5931623.59619088045718485257533595084_5.51.483.6267.38091.600304.7903686.54470604593404454865527111473_6.81.633.2711.11281.129374.1990705.14561065729273174530359048636.dcm
data/1.10.177.3768.96542.994488.5287730.66175790315407691665796802664_3.66.778.6977.18536.121149.6318540.99045988055389082929143350416_7.45.258.3465.52388.141008.9966840.98253625347272363336564989358.dcm
data/1.10.183.5299.67385.778599.6430679.21645327620642564768799820616_9.87.917.4029.15611.143622.1993344.58242060787312593373527431002_1.14.109.7930.48815.296542.4208170.48240616080978605316495116527.dcm
data/1.10.192.7017.79057.441390.2489956.46406478760041391051195435077_1.86.383.1460.90462.687091.1040961.48412696957170806939220702824_7.40.407.3678.72494.531998.9114190.84130181989755734414551037105.dcm
data/1.10.258.6620.89733.276845.5984942.63762948182850655811542524982_4.57.312.6848.94429.792320.3873319.78188965844694291015030262611_5.17.104.1148.34368.106247.9683090.10909107384180

In [7]:
# Downloading Data from S3

import boto3
import botocore
from boto3 import client
import os
os.mkdir("data")
os.mkdir("labels")
# os.mkdir("PNEUMONIA")
# os.mkdir("NORMAL")
BUCKET_NAME = bucket
c = 0
conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
for key in conn.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])
    KEY = key['Key']
    s3 = boto3.resource('s3')
    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, f"{key['Key']}")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
    c = c+1


FileExistsError: [Errno 17] File exists: 'data'

In [8]:
# Converting downloaded data to png

# Dcm to Png
!pip install pydicom
import pydicom as dicom
import pydicom
import json
import os
from pathlib import Path
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom import dcmread
import os
import pickle
from PIL import Image
import numpy as np
from PIL import Image
import PIL

def read_xray(path, voi_lut=True, fix_monochrome=True):
    try:
        print("Converting to PNG .........................")
        dicom = dcmread(path, force=True)
        print(dicom.SOPInstanceUID, ">>>>>>", dicom.InstanceNumber, ">>>>>", dicom.SeriesInstanceUID)
        #if voi_lut:
        if voi_lut and len(dicom.get("VOILUTSequence", [])):
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array
        if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
        data = data - np.min(data)
        data = data / np.max(data)
        data = (data * 255).astype(np.uint8)
        return data
    except Exception as e:
        print(e, ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        return "corrupt"

Keyring is skipped due to an exception: 'keyring.backends'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.1 MB/s eta 0:00:0000:01


In [9]:
files_ = Path("data").rglob('*.dcm')
print(files_)
c = 0
for i in files_:
    try:
        print(i)
        img = read_xray(i)
        with open("filename.pkl", 'wb') as f:
            pickle.dump(img, f)
        ims = pickle.load(open("filename.pkl", "rb"))
        norm = (ims.astype(np.float) - ims.min()) * 255.0 / (ims.max() - ims.min())
        Image.fromarray(norm.astype(np.uint8)).save(f"training_data_png/{c}.png")
        c = c + 1
        print(c, "Done")
    except Exception as e:
        print(e)

<generator object Path.rglob at 0x7f6b1f4e2ed0>
data/7.33.322.8202.66705.661405.4106430.51801910620212574810042039688_8.37.917.1061.18335.804654.9969916.57399252307428188864073390884_9.36.163.6335.96528.958775.2025579.13466544142149642138760107013.dcm
Converting to PNG .........................
9.36.163.6335.96528.958775.2025579.13466544142149642138760107013 >>>>>> None >>>>> 8.37.917.1061.18335.804654.9969916.57399252307428188864073390884


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()


[Errno 2] No such file or directory: 'training_data_png/0.png'
data/1.32.492.6222.81503.413933.5900548.67495136899513705605242805471_7.80.389.3802.19074.474788.3415957.14938570290039521831822956034_1.12.277.4272.84780.325732.1730891.74706905275212944011113903363.dcm
Converting to PNG .........................
1.12.277.4272.84780.325732.1730891.74706905275212944011113903363 >>>>>> None >>>>> 7.80.389.3802.19074.474788.3415957.14938570290039521831822956034
[Errno 2] No such file or directory: 'training_data_png/0.png'
data/2.77.228.9146.47949.121489.6944699.89552851552200162928736566692_5.14.607.4701.12415.923827.5417494.67457611002174379956354989723_1.12.678.3431.77290.199682.3847611.82656095123468839888919818365.dcm
Converting to PNG .........................
1.12.678.3431.77290.199682.3847611.82656095123468839888919818365 >>>>>> None >>>>> 5.14.607.4701.12415.923827.5417494.67457611002174379956354989723
[Errno 2] No such file or directory: 'training_data_png/0.png'
data/4.96.875.3316.

## Data
### Getting the data



### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [10]:
inputs = sagemaker_session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

NameError: name 'prefix' is not defined

## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [13]:
!pygmentize code/classifier.py

import argparse
import json
import logging
import os
import sys
#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

import shlex, subprocess
# def install(package):
#     os.system("pip install " +  package)
    
# install('pillow')
# install('requests')
# install('pydicom')
from PIL import Image
import requests
import pydicom

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def get_pretrained_model(model_name):
    """Retrieve a pre-trained model from torchvision

    Params
    -------
        model_name (str): name of the model (currently only accepts vgg16 and resnet50)

    Return
    --------
        model (PyTorch model): cnn

    """

    if model_name == 'vgg16':
        model = models.vgg16(pretr

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [9]:
!pip install --upgrade sagemaker==2.110.0

In [12]:
from sagemaker.pytorch import PyTorch


In [13]:
!pip show sagemaker | grep Version

Version: 2.120.0


In [16]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="classifier.py",
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=1,
    instance_type="ml.c5.2xlarge",
    # instance_type="ml.g4dn.xlarge",
    hyperparameters={"epochs": 1, "backend": "gloo"},
    dependencies=['code/requirements.txt'],
    source_dir = "code",
    max_run=20,
    
)

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [17]:
estimator.fit({"training":"s3://xray-nano/nano/train/" ,"testing":"s3://xray-nano/nano/test/","validating":"s3://xray-nano/nano/val/"})

2023-01-10 21:03:59 Starting - Starting the training job...
2023-01-10 21:04:15 Starting - Preparing the instances for trainingProfilerReport-1673384639: InProgress
.........
2023-01-10 21:05:56 Downloading - Downloading input data...
2023-01-10 21:06:19 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-01-10 16:06:20,042 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-01-10 16:06:20,043 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-10 16:06:20,045 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-01-10 16:06:20,053 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-01-10 16:06:20,055 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-01-10 16:06:

## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [18]:
estimator.__dict__


{'framework_version': '1.11.0',
 'py_version': 'py38',
 'role': 'arn:aws:iam::023180687239:role/service-role/AmazonSageMaker-ExecutionRole-20220906T142944',
 'instance_count': 1,
 'instance_type': 'ml.c5.2xlarge',
 'keep_alive_period_in_seconds': None,
 'instance_groups': None,
 'volume_size': 30,
 'volume_kms_key': None,
 'max_run': 20,
 'input_mode': 'File',
 'metric_definitions': None,
 'model_uri': None,
 'model_channel_name': 'model',
 'code_uri': None,
 'code_channel_name': 'code',
 'source_dir': 'code',
 'git_config': None,
 'container_log_level': 20,
 '_hyperparameters': {'epochs': 1,
  'backend': 'gloo',
  'sagemaker_submit_directory': 's3://sagemaker-ap-south-1-023180687239/pytorch-training-2023-01-10-21-03-59-165/source/sourcedir.tar.gz',
  'sagemaker_program': 'classifier.py',
  'sagemaker_container_log_level': 20,
  'sagemaker_job_name': 'pytorch-training-2023-01-10-21-03-59-165',
  'sagemaker_region': 'ap-south-1'},
 'code_location': None,
 'entry_point': 'classifier.py',

In [19]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.2xlarge")

-----!

### Evaluate

You can use the test images to evalute the endpoint. The accuracy of the model depends on how many it is trained. 

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [20]:
predictor.__dict__

{'endpoint_name': 'pytorch-training-2023-01-10-21-08-42-301',
 'sagemaker_session': <sagemaker.session.Session at 0x7f6b30991650>,
 'serializer': <sagemaker.serializers.NumpySerializer at 0x7f6b1f3f62d0>,
 'deserializer': <sagemaker.deserializers.NumpyDeserializer at 0x7f6b1f3f6310>,
 '_endpoint_config_name': None,
 '_model_names': None,
 '_context': None}

In [21]:
predictor.endpoint_name 

'pytorch-training-2023-01-10-21-08-42-301'

In [22]:
import boto3

client = boto3.client('sagemaker-runtime')

custom_attributes = "c000b4f9-df62-4c85-a0bf-7c525f9104a4"  # An example of a trace ID.
endpoint_name = predictor.endpoint_name                                        # Your endpoint name.
content_type = "application/json"                                        # The MIME type of the input data in the request body.
accept = "application/json"                                              # The desired MIME type of the inference in the response.
payload = json.dumps({"url":"https://storage.googleapis.com/kaggle-datasets-images/17810/23340/c8372ebbe20b0f671c2f3c501ba51412/dataset-cover.jpeg?t=2018-03-24-19-05-18"})                                           # Payload for inference.
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    CustomAttributes=custom_attributes, 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

print(response)   

{'ResponseMetadata': {'RequestId': 'a5f2ebac-7b6c-44a9-838c-490a31d571f6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a5f2ebac-7b6c-44a9-838c-490a31d571f6', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Tue, 10 Jan 2023 21:14:47 GMT', 'content-type': 'application/json', 'content-length': '66'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f6b1ff16750>}


In [23]:
from pprint import pprint

In [24]:
pprint(response) 

{'Body': <botocore.response.StreamingBody object at 0x7f6b1ff16750>,
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '66',
                                      'content-type': 'application/json',
                                      'date': 'Tue, 10 Jan 2023 21:14:47 GMT',
                                      'x-amzn-invoked-production-variant': 'AllTraffic',
                                      'x-amzn-requestid': 'a5f2ebac-7b6c-44a9-838c-490a31d571f6'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'a5f2ebac-7b6c-44a9-838c-490a31d571f6',
                      'RetryAttempts': 0}}


In [25]:
r = json.load(response["Body"])

In [26]:
r

{'response': {'findings': [{'name': 'class', 'probability': 54}]}}